# use activate dato-env command

In [1]:
import graphlab as gl
from graphlab import SFrame
import os
from dateutil.parser import parse as dateutil_parse
from six.moves import cPickle as pickle
import time
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler as skStandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.cross_validation import train_test_split,cross_val_score

PATH = 'season_1/'
CLEAN_PATH = PATH+'clean/'

from mylib import myStandardScaler,process_order,process_traffic,get_order_group,get_traffic_group,XY_order_traffic, prediction2submit, Search_Model, DISTRICTS, mymetrics

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
cluster_map = pd.read_csv(CLEAN_PATH+'cluster_map.csv',index_col=0)
poi = pd.read_csv(CLEAN_PATH+'poi.csv',index_col=0)
train_order_group = pd.read_pickle(CLEAN_PATH+'train_order_group.pickle')
test_order_group = pd.read_pickle(CLEAN_PATH+'test_order_group.pickle')
train_traffic_group = pd.read_pickle(CLEAN_PATH+'train_traffic_group.pickle')
test_traffic_group = pd.read_pickle(CLEAN_PATH+'test_traffic_group.pickle')
test_target = pd.read_csv(CLEAN_PATH+'test_target.csv',index_col=0,parse_dates=True)

In [3]:
train_slot = pd.Index(sorted(train_order_group.values()[0].index.unique()))
train_slot = pd.Index(filter(lambda x: x%1000 >4,train_slot))
test_slot = test_target['datetimeslot']

In [4]:
now = time.time()

train_XY_group = dict()
for district in DISTRICTS:
    train_XY_group[district] = XY_order_traffic(district,train_order_group,train_traffic_group,train_slot)
test_XY_group = dict()
for district in DISTRICTS:
    test_XY_group[district] = XY_order_traffic(district,test_order_group,test_traffic_group,test_slot)
for district in DISTRICTS:
    scaler = myStandardScaler()
    train_XY_group[district][0] = scaler.fit_transform(train_XY_group[district][0])
    test_XY_group[district][0] = scaler.transform(test_XY_group[district][0])
    
stop = time.time()
print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)

Take 00:00:16


In [5]:
all_now = time.time()
test_prediction = dict()
for district in DISTRICTS:
    now = time.time()
    print 'Searching %s...'%district
    trainSet = SFrame(train_XY_group[district][0])
    targetSet = SFrame(train_XY_group[district][1])
    trainSet = trainSet.add_columns(targetSet)
    model = gl.boosted_trees_regression.create(trainSet, target='X1', features=train_XY_group[district][0].columns,
                                          max_iterations=5000, validation_set='auto', max_depth=5, step_size=0.03, 
                                          min_loss_reduction=1.0, min_child_weight=0.1, row_subsample=1.0, column_subsample=1.0, verbose=False, 
                                          random_seed=1, metric='auto')
    sPredict = model.predict(SFrame(test_XY_group[district][0]))
    myPredict = pd.Series(sPredict.to_numpy(), index = test_XY_group[district][0].index)
    test_prediction[district] = myPredict - test_XY_group[district][1].fillna(0)
    with open(CLEAN_PATH+'prediction_gl/test_prediction_%s.pickle'%(district),'wb') as f:
        pickle.dump(test_prediction[district],f)
    print "The metrics:", mymetrics(pd.Series(model.predict(SFrame(train_XY_group[district][0])).to_numpy()), pd.Series(trainSet['X1'].to_numpy()))
    stop = time.time()
    print 'Take %02d:%02d:%02d' % ((stop-now)/3600,(stop-now)/60,(stop-now)%60)
all_stop = time.time()
print 'Totally take %02d:%02d:%02d' % ((all_stop-all_now)/3600,(all_stop-all_now)/60,(all_stop-all_now)%60)

Searching f47f35242ed40655814bc086d7514046...
This trial license of GraphLab Create is assigned to czwjovi@foxmail.com and will expire on June 26, 2016. Please contact trial@dato.com for licensing options or to request a free non-commercial license for personal or academic use.


2016-05-29 09:54:20,839 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: C:\Users\ALIENW~1\AppData\Local\Temp\graphlab_server_1464486859.log.0


The metrics: 0.87355117354
Take 00:00:48
Searching 38d5ad2d22b61109fd8e7b43cd0e8901...
The metrics: 0.243694949205
Take 00:00:49
Searching 364bf755f9b270f0f9141d1a61de43ee...
The metrics: 0.563832897464
Take 00:00:45
Searching 49ac89aa860c27e26c0836cb8dab2df2...
The metrics: 1.74463808513
Take 00:00:45
Searching 91690261186ae5bee8f83808ea1e4a01...
The metrics: 0.451055501128
Take 00:00:48
Searching 8bb37d24db1ad665e706c2655d9c4c72...
The metrics: 0.95636014652
Take 00:00:44
Searching a5609739c6b5c2719a3752327c5e33a7...
The metrics: 0.505155793015
Take 00:00:47
Searching 08232402614a9b48895cc3d0aeb0e9f2...
The metrics: 1.29332172779
Take 00:00:44
Searching b702e920dcd2765e624dc1ce3a770512...
The metrics: 0.515048215592
Take 00:00:47
Searching 52d7b69796362a8ed1691a6cc02ddde4...
The metrics: 1.11371065303
Take 00:00:46
Searching 62afaf3288e236b389af9cfdc5206415...
The metrics: 0.168154651874
Take 00:00:48
Searching 445ff793ebd3477d4a2e0b36b2db9271...
The metrics: 1.48296086554
Take 00:00

In [40]:
submit = prediction2submit(test_prediction, cluster_map)

In [41]:
def ceil_gap(x):
    if(x < 1):
        res = 1
    elif(x < 2 and x > 1.5):
        res = 2
    else:
        res = x
    return res

In [42]:
newgap = submit['prediction'].apply(lambda x: ceil_gap(x))
submit['gap2'] = newgap
submit = submit.drop('prediction', axis = 1)

In [43]:
submit

district             dts        gap2
datetimeslot                                      
20160122046          1   2016-01-22-46    7.780203
20160122046          2   2016-01-22-46    1.000000
20160122046          3   2016-01-22-46    1.000000
20160122046          4   2016-01-22-46    5.167943
20160122046          5   2016-01-22-46    1.000000
20160122046          6   2016-01-22-46    9.287977
20160122046          7   2016-01-22-46   46.458427
20160122046          8   2016-01-22-46  336.013763
20160122046          9   2016-01-22-46    2.000000
20160122046         10   2016-01-22-46    2.908944
20160122046         11   2016-01-22-46    1.000000
20160122046         12   2016-01-22-46   27.751083
20160122046         13   2016-01-22-46    1.000000
20160122046         14   2016-01-22-46   62.784023
20160122046         15   2016-01-22-46    1.000000
20160122046         16   2016-01-22-46    1.157668
20160122046         17   2016-01-22-46    1.000000
20160122046         18   2016-01-22-46    2.000000
20160122046         19   2016-01-22-46    1.472654
20160122046         20   2016-01-22-46    4.413148
20160122046         21   2016-01-22-46    5.486962
20160122046         22   2016-01-22-46    6.268541
20160122046         23   2016-01-22-46  218.019485
20160122046         24   2016-01-22-46  172.395676
20160122046         25   2016-01-22-46    1.153227
20160122046         26   2016-01-22-46    8.322971
20160122046         27   2016-01-22-46   14.761171
20160122046         28   2016-01-22-46    9.029046
20160122046         29   2016-01-22-46    3.600707
20160122046         30   2016-01-22-46    1.000000
...                ...             ...         ...
20160130142         37  2016-01-30-142    1.400180
20160130142         38  2016-01-30-142    1.000000
20160130142         39  2016-01-30-142    3.422405
20160130142         40  2016-01-30-142    1.177429
20160130142         41  2016-01-30-142    1.000000
20160130142         42  2016-01-30-142    2.028037
20160130142         43  2016-01-30-142    1.000000
20160130142         44  2016-01-30-142    1.000000
20160130142         45  2016-01-30-142    1.000000
20160130142         46  2016-01-30-142    1.000000
20160130142         47  2016-01-30-142    2.687706
20160130142         48  2016-01-30-142    8.327179
20160130142         49  2016-01-30-142    1.000000
20160130142         50  2016-01-30-142    1.000000
20160130142         51  2016-01-30-142  164.704498
20160130142         52  2016-01-30-142    1.000000
20160130142         53  2016-01-30-142    1.000000
20160130142         54  2016-01-30-142    1.109751
20160130142         55  2016-01-30-142    1.000000
20160130142         56  2016-01-30-142    1.000000
20160130142         57  2016-01-30-142    1.000000
20160130142         58  2016-01-30-142    1.000000
20160130142         59  2016-01-30-142    1.000000
20160130142         60  2016-01-30-142    1.000000
20160130142         61  2016-01-30-142    1.000000
20160130142         62  2016-01-30-142    1.000000
20160130142         63  2016-01-30-142    1.000000
20160130142         64  2016-01-30-142    1.000000
20160130142         65  2016-01-30-142    1.000000
20160130142         66  2016-01-30-142    1.000000

[2838 rows x 3 columns]

In [44]:
submit.to_csv(PATH+'submit/gl_boosted_tree_prediction_max_depth_3_ceil2.csv',index=None,header=False)